In [1]:
# Notebook til at sammenholde vaccinationsdata med tilfælde
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]


saveFigures is set to: True
Done loading packages


In [2]:
# Get most recent vaccination directory
ssidatapath = "ssi_vacc"
rootdir = os.getcwd() +"\\" + ssidatapath

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])

# print(latestdir)
# print(latestDate)

# Load vaccination data
vaccFile = '\\Vaccine_DB\\Vaccinationer_region_aldgrp_koen.csv'

curvaccdf = pd.read_csv(latestdir+vaccFile,delimiter=';',encoding='latin1')
curvaccdf.tail()

,Regionsnavn,Aldersgruppe,sex,Antal første vacc.,Antal faerdigvacc.
95,Syddanmark,70-79,M,60789,60429
96,Syddanmark,80-89,K,31454,31265
97,Syddanmark,80-89,M,23945,23818
98,Syddanmark,90+,K,7142,7034
99,Syddanmark,90+,M,3112,3072


In [3]:
curFileName = '\\Regionalt_DB\\05_bekraeftede_tilfaelde_doede_pr_region_pr_alders_grp.csv'
allvaccdfs = []
allvaccDates = []

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    try:
        curDate = pd.to_datetime(curPath[-10:])

        
        curcsvpath = curPath + vaccFile
        
        curdf = pd.read_csv(curcsvpath,delimiter = ',',dtype=str,encoding='latin-1')
        
        if (len(curdf.columns) == 1):
            curdf = pd.read_csv(curcsvpath,delimiter = ';',dtype=str,encoding='latin-1')
        
        # if (curDate  > np.datetime64('2021-07-12')):
        #     curdf = pd.read_csv(curcsvpath,delimiter = ';',dtype=str,encoding='latin-1')
        # else:
        #     curdf = pd.read_csv(curcsvpath,delimiter = ',',dtype=str,encoding='latin-1')

        allvaccDates.append(curDate)
        allvaccdfs.append(curdf)
    except:
        print(f'Problem with {curPath}')
    
print(f'Number of dataframes in allvaccdfs: {len(allvaccdfs)}')

Problem with c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-02-19
Problem with c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-02-20
Problem with c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-02-21
Problem with c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-02-22
Problem with c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-02-23
Problem with c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-02-25
Problem with c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_vacc\SSI_vacc_2021-04-28
Number of dataframes in allvaccdfs: 153


In [4]:
# allvaccdfs[132]
# allvaccDates[132] > np.datetime64('2021-07-12')
# len(allvaccdfs[126].columns)
# allvaccDates[126]
allvaccdfs[130].columns

Index(['Regionsnavn', 'Aldersgruppe', 'sex', 'Antal første vacc.',
       'Antal faerdigvacc.'],
      dtype='object')

In [5]:
# Collect cases and deaths in dataframes
numVaccDataframe = len(allvaccdfs)

# Whole country, age-split
allAge = curvaccdf['Aldersgruppe'].unique()

dfVacc = pd.DataFrame()
dfVaccFull = pd.DataFrame()
dfVacc['Dato'] = allvaccDates
dfVaccFull['Dato'] = allvaccDates

for ag in range(len(allAge)):
    curAge = allAge[ag]

    allOnce = []
    allFull = []

    prevDate = allvaccDates[0]

    for da in range(0,numVaccDataframe):
        thisdf = allvaccdfs[da]
        thisdf = thisdf[thisdf['Aldersgruppe'] == curAge]

        thisOnce = thisdf['Antal første vacc.'].astype(int).sum()
        thisFull = thisdf['Antal faerdigvacc.'].astype(int).sum()


        allOnce.append(thisOnce)
        allFull.append(thisFull)

    dfVacc[curAge] = allOnce
    dfVaccFull[curAge] = allFull

In [6]:
# dfVaccFull

In [7]:
# Load cases-data
dashDir = 'ssi_dashboard/'
rootdir = os.getcwd() +"\\" + dashDir

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])
    
curFileName = '\\Regionalt_DB\\05_bekraeftede_tilfaelde_doede_pr_region_pr_alders_grp.csv'
alldfs = []
allDates = []

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    try:
        curDate = pd.to_datetime(curPath[-10:])

        
        curcsvpath = curPath + curFileName
        curdf = pd.read_csv(curcsvpath,delimiter = ';',dtype=str,encoding='latin-1')

        allDates.append(curDate)
        alldfs.append(curdf)
    except:
        print(curPath)
    
print(f'Number of dataframes in alldfs: {len(alldfs)}')

c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_dashboard/SSI_dashboard_2021-05-25
Number of dataframes in alldfs: 96


In [8]:
# Collect cases and deaths in dataframes
numDataframe = len(alldfs)

# Whole country, age-split
allAge = curdf['Alders gruppe'].unique()

dfCase = pd.DataFrame()
dfDeath = pd.DataFrame()

dfCase['Dato'] = allDates
dfDeath['Dato'] = allDates

for ag in range(len(allAge)):
    curAge = allAge[ag]

    allCase = []
    allDeaths = []

    prevDate = allDates[0]

    for da in range(0,numDataframe):
        thisdf = alldfs[da]
        thisdf = thisdf[thisdf['Alders gruppe'] == curAge]

        thisCase = thisdf['Bekræftede tilfælde'].astype(int).sum()
        thisDeaths = thisdf['Døde'].astype(int).sum()


        allCase.append(thisCase)
        allDeaths.append(thisDeaths) 

    dfCase[curAge] = allCase
    dfDeath[curAge] = allDeaths

In [9]:
# Calculate difference, and average out weekends
dfCaseDiff = dfCase.diff().iloc[1:]
dfDeathDiff = dfDeath.diff().iloc[1:]
# dfCaseDiff

actualDiffDates = dfCaseDiff.Dato
actualDiffDates = (actualDiffDates/np.timedelta64(1,'D')).astype(int)

# actualDiffDates

fullDates = np.arange(allDates[0],allDates[-1],np.timedelta64(1,'D'))

dfCaseDiffCorr = dfCaseDiff.copy()
dfDeathDiffCorr = dfDeathDiff.copy()

for r in range(0,len(dfCaseDiff)):
    if ((actualDiffDates.values[r]) != 1):
        # print(dfCaseDiff.iloc[r])
        curRow = dfCaseDiff.iloc[r]
        numDays = int(curRow.Dato/np.timedelta64(1,'D'))

        rowToAdd = curRow/numDays

        for k in range(1,numDays+1):
            dfCaseDiffCorr.loc[r+(0.1*k)] = rowToAdd
        dfCaseDiffCorr = dfCaseDiffCorr.drop(r+1)
        
        # And deaths
        curRow = dfDeathDiff.iloc[r]
        numDays = int(curRow.Dato/np.timedelta64(1,'D'))

        rowToAdd = curRow/numDays

        for k in range(1,numDays+1):
            dfDeathDiffCorr.loc[r+(0.1*k)] = rowToAdd
        dfDeathDiffCorr = dfDeathDiffCorr.drop(r+1)

        # newdf = pd.concat([dfCaseDiff.iloc[:2], curRow, dfCaseDiff.iloc[2:]]).reset_index(drop=True)
        


dfCaseDiffCorr = dfCaseDiffCorr.sort_index().reset_index(drop=True)
dfDeathDiffCorr = dfDeathDiffCorr.sort_index().reset_index(drop=True)
# dfCaseDiffCorr
dfCaseDiffCorr = dfCaseDiffCorr.iloc[:,1:]
dfDeathDiffCorr = dfDeathDiffCorr.iloc[:,1:]

# Make plots

In [10]:
# Test plot
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

curAgeGroup = '50-59'

ax1.plot(fullDates,dfCaseDiffCorr[curAgeGroup],'k.:',markersize=3,linewidth=0.5)
ax1.plot(rnTime(fullDates,7),rnMean(dfCaseDiffCorr[curAgeGroup],7),'k')
ax1.set_ylim(bottom=0)

ax2.fill_between(allvaccDates,dfVacc[curAgeGroup],color='xkcd:light green')
ax2.fill_between(allvaccDates,dfVaccFull[curAgeGroup],color='g')
ax2.set_ylim(bottom=0)

ax1.set_ylabel('Tilfælde')
ax2.set_ylabel('Vaccinerede')

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
dfAges = pd.read_excel('DKfolketal2021clean.xlsx')
DK_age = []
DK_age.append(dfAges.iloc[0:10].Antal.sum())
DK_age.append(dfAges.iloc[10:20].Antal.sum())
DK_age.append(dfAges.iloc[20:30].Antal.sum())
DK_age.append(dfAges.iloc[30:40].Antal.sum())
DK_age.append(dfAges.iloc[40:50].Antal.sum())
DK_age.append(dfAges.iloc[50:60].Antal.sum())
DK_age.append(dfAges.iloc[60:70].Antal.sum())
DK_age.append(dfAges.iloc[70:80].Antal.sum())
DK_age.append(dfAges.iloc[80:90].Antal.sum())
DK_age.append(dfAges.iloc[90:].Antal.sum())
DK_age = np.array(DK_age)


In [12]:
dfVaccNone = DK_age * np.ones(dfVacc.iloc[:,1:].shape) - dfVacc.iloc[:,1:]
# dfVaccNone
# dfVacc
# dfVaccNorm = dfVacc.iloc[:,1:].div(DK_age)
# dfVaccFullNorm = dfVaccFull.iloc[:,1:].div(DK_age)

In [13]:
fig,ax1 = plt.subplots()
ax1.plot(allvaccDates,dfVaccNone['90+'])
ax1.plot(allvaccDates,dfVaccNone['80-89'])
ax1.plot(allvaccDates,dfVaccNone['70-79'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Comparison plot

for curAgeGroup in dfCaseDiffCorr.columns:
    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
    # curAgeGroup = '30-39'

    ax1.plot(fullDates,dfCaseDiffCorr[curAgeGroup],'k.:',markersize=3,linewidth=0.5)
    ax1.plot(rnTime(fullDates,7),rnMean(dfCaseDiffCorr[curAgeGroup],7),'k')
    ax1.set_ylim(bottom=0)

    # ax2.fill_between(allvaccDates,dfVaccFull[curAgeGroup]+dfVaccNone[curAgeGroup],color='g')

    # ax2.fill_between(allvaccDates,dfVacc[curAgeGroup]+dfVaccNone[curAgeGroup],color='xkcd:light green')
    # # ax2.fill_between(allvaccDates,dfVacc[curAgeGroup]+dfVaccNone[curAgeGroup],color='g')
    # ax2.fill_between(allvaccDates,dfVaccNone[curAgeGroup],color='k')

    dfVaccOnlyOnce = dfVacc - dfVaccFull 
    ax2.fill_between(allvaccDates,dfVaccNone[curAgeGroup]+dfVaccOnlyOnce[curAgeGroup]+dfVaccFull[curAgeGroup],color='g')
    ax2.fill_between(allvaccDates,dfVaccNone[curAgeGroup]+dfVaccOnlyOnce[curAgeGroup],color='xkcd:light green')
    ax2.fill_between(allvaccDates,dfVaccNone[curAgeGroup],color='grey')
    ax2.set_ylim(bottom=0)
    ax2.set_ylim(top=(dfVaccNone[curAgeGroup]+dfVaccOnlyOnce[curAgeGroup]+dfVaccFull[curAgeGroup])[0]) # Full population


    ax1.set_ylabel('Tilfælde')
    ax2.set_ylabel('Vaccinerede')
    
    ax1.set_title(curAgeGroup)

    ax1.set_xlim([fullDates[0],fullDates[-1]])
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B')) 

    plt.tight_layout()
    
    if saveFigures:
        plt.savefig('figsVacc/CasesAndVaccinations_'+curAgeGroup)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
dfCaseDiffCorr

,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
0,77.000000,82.000000,85.000000,76.000000,80.000000,65.000000,42.000000,14.000000,4.000000,2.000000
1,76.000000,274.000000,406.000000,326.000000,329.000000,280.000000,194.000000,91.000000,28.000000,3.000000
2,87.000000,137.000000,146.000000,128.000000,124.000000,116.000000,72.000000,41.000000,11.000000,0.000000
3,59.000000,99.000000,120.000000,94.000000,93.000000,80.000000,61.000000,24.000000,16.000000,-1.000000
4,67.666667,99.333333,104.666667,85.000000,89.333333,78.000000,34.666667,23.666667,6.333333,0.000000
5,67.666667,99.333333,104.666667,85.000000,89.333333,78.000000,34.666667,23.666667,6.333333,0.000000
6,67.666667,99.333333,104.666667,85.000000,89.333333,78.000000,34.666667,23.666667,6.333333,0.000000
7,74.000000,129.000000,153.000000,127.000000,113.000000,90.000000,46.000000,32.000000,7.000000,0.000000
8,85.000000,135.000000,136.000000,117.000000,114.000000,107.000000,55.000000,24.000000,12.000000,2.000000
9,102.000000,156.000000,145.000000,110.000000,103.000000,95.000000,46.000000,24.000000,5.000000,0.000000
